In [72]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# @Author: Mingyeong Yang (mmingyeong@kasi.re.kr)
# @Date: 2024-07-03
# @Filename: 240703_shon_lecture.ipynb
# This project was conducted at the 2024 KIAS Summer School on Extragalactics and Cosmology.

In [73]:
from time import perf_counter as clock
from pathlib import Path
from urllib import request
import glob

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util
import ppxf.sps_util as lib

In [74]:
# Basic pacakges 
import pandas as pd
import numpy  as np
import datetime

import os 
import h5py

from   pathlib import Path
from   time    import perf_counter as clock

# astropy
from   astropy.table import Table
import astropy.units as u
from   astropy.coordinates import SkyCoord

# For Reading and manipulating spectra
from   astropy.io   import fits
from   scipy.signal import savgol_filter

# For pPXF
from   ppxf.ppxf      import ppxf
import ppxf.ppxf_util as util
import ppxf.sps_util  as lib

## For plotting 
import matplotlib        as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from   matplotlib        import rc
from   matplotlib        import cm
from   matplotlib.colors import LogNorm
from   matplotlib.font_manager import FontProperties
import matplotlib.colors as mcolors

In [75]:
# A Simple subroutine for checking if you have spectra in your directory
def file_exists(directory, filename, use_pathlib=False):
    if use_pathlib:
        directory_path = Path(directory)
        file_path = directory_path / filename
        return file_path.exists()
    else:
        file_path = os.path.join(directory, filename)
        return os.path.exists(file_path)

### 계산에 필요한 물리상수를 정의함.

In [76]:
# Some numerical constants 
c    = 2.99792e5
tol  = 1./3600.
dtor = np.pi / 180.

# For smoothing the spectra
window_length = 21 # Window length must be a positive odd integer
polyorder     =  3 # Polynomial order must be less than window_length

# Some common parameters for plotting
LM = 0.16
RM = 0.95
BM = 0.13
TM = 0.97

plt.rc('axes',  labelsize=35)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)

plt.rcParams['axes.linewidth'] = 3
plt.rcParams['xtick.top']   = True
plt.rcParams['ytick.right'] = True

tick_typs  = ['xtick', 'ytick']
tick_items = ['direction', 'major.size', 'major.width', 'minor.visible', 'minor.size', 'minor.width']
tick_vals  = ['in', 12, 2, True, 6, 2]

for tick_typ in tick_typs:
    for i, tick_item in enumerate(tick_items):
        plt.rcParams[tick_typ + '.' + tick_item] = tick_vals[i]

In [77]:
# Working directories
## Choose your own directory
spedir = "C:/Users/mming/Desktop/240701_KIAS_Summerschool_외부은하와우주론여름학교/GalaxySpectrum_손주비/SDSS/"

In [78]:
# READ the F2 table
tab   = pd.read_csv('F2_test.csv')
narr  = len(tab['OBJID'])
petrm = tab['PETROMAG_R'] - tab['EXTINCTION_R']

sel = (petrm > 14.5) & (petrm < 17.5) & (tab['Z_SDSS_SPECFILE'] != 'NN')
dum =  petrm[sel]
print(len(dum))

f2coords = SkyCoord(ra=tab['RA'], dec=tab['DEC'], unit=(u.deg, u.deg))

C:\Users\mming\AppData\Local\Temp\ipykernel_33496\534442019.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  tab   = pd.read_csv('F2_test.csv')


566


In [79]:
# Select the brightest galaxy spectrum
sel = (tab['Z_SDSS_SPECFILE'] != 'NN') & (petrm > 15.0) & (petrm < 16.0) & (tab['Z'] > 0.05) & (tab['Z'] < 0.10)
dum =  petrm[sel]
print(len(dum))
print(tab['Z_SDSS_SPECFILE'][sel])
print(tab['Z'][64248], tab['ZERR'][64248])

4
14447    spec-1938-53379-0110.fits
61678    spec-1937-53388-0453.fits
64248    spec-1938-53379-0053.fits
82275    spec-1937-53388-0577.fits
Name: Z_SDSS_SPECFILE, dtype: object
0.0627988576889038 4.395649739308283e-05


In [80]:
spedir

'C:/Users/mming/Desktop/240701_KIAS_Summerschool_외부은하와우주론여름학교/GalaxySpectrum_손주비/SDSS/'

### Let's run the pPXF to derive the velocity dispersions 
### 속도 분산을 구해보자

In [81]:
# HW #1 
# Let's try to run the pPXF to compute the redshifts and velocity dispersions for the remaining F2 galaxies 
# Compare your results with the Portsmouth measurements.
# 나머지 F2 은하의 적색편이와 속도 분산을 계산하기 위해 pPXF를 실행해 보겠습니다.
# 결과를 포츠머스 측정값과 비교합니다.

In [84]:
import os

# FITS 파일 목록
file_list = [
    'spec-1938-53379-0110.fits',
    'spec-1937-53388-0453.fits',
    'spec-1938-53379-0053.fits',
    'spec-1937-53388-0577.fits'
]

# 출력을 저장할 파일 경로
output_file = 'output_log.txt'

# 파일을 열어 쓰기 모드로 엽니다.
with open(output_file, 'w') as f:
        
        for file in file_list:
                print(f"{file} processing start\n")
                f.write("----------------------------------------------------------------\n")
                f.write(f"{file} processing start\n")
                f.write(f"{file} processing start\n")
                f.write(f"{file} processing start\n")
                ## READ the spectrum
                name = os.path.basename(file)
                filename_without_extension = os.path.splitext(name)[0]
                spe      = fits.open(spedir + file)
                spedata  = spe[1].data
                spehdr   = spe[1].header
                spez     = 0.06280
                spezerr  = 4.39565e-5

                lnwave   = spedata['loglam'] * np.log(10.) # Convert lg --> ln
                wave     = np.exp(lnwave)                   # Wavelength in Angstroms

                flux =  spedata['flux']
                nflux = flux / np.median(flux) # Normalize spectrum to avoid numerical issues
                sflux = savgol_filter(nflux, window_length, polyorder) # Smoothed spectrum

                ## Define the page
                #fig, axs = plt.subplots(1, 1, figsize=(12, 6))
                #fig.subplots_adjust(left=LM, right=RM, bottom=BM, top=TM) 

                #axs.set_xlabel('rest-frame wavelength')
                #axs.set_ylabel('Flux')
                #axs.set_xlim(3500., 7000.)

                ## Define Axes Styles
                #axs.plot(wave, nflux, c='k')
                #axs.plot(wave, sflux, c='r')
                #axs.set_title(f'Spectrum Analysis for {filename_without_extension}')
                #plt.savefig(f"{filename_without_extension}.png", format='png', dpi=300)

                
                ## Define pPXF models
                sps_name = 'emiles' # 'fsps', 'galaxev', 'xsl' - choose the model you want

                ppxf_dir = Path(lib.__file__).parent
                basename = f"spectra_{sps_name}_9.0.npz"

                filename = ppxf_dir / 'sps_models' / basename
                wave_range_temp = [3500., 10000.]


                # Manipulate the spectrum and wavelengths
                dlnwave  = (lnwave[-1] - lnwave[0]) / (lnwave.size - 1)   # Compute the Delta Lambda / Use full lam range for accuracy
                velscale = c * dlnwave                                    # Velocity scale in km/s per pixel (eq.8 of Cappellari 2017)
                noise    = np.full_like(nflux, 0.0166)                    # Assume constant noise per pixel here - JS, where does this 0.0166 comes from?

                dwave    = np.gradient(wave)
                wdisp    = spedata['wdisp']                              # Intrinsic dispersion of every pixel, in pixels units, wavelength dispersion in pixel=dloglam units
                fwhm     = 2.3555 * wdisp * dwave

                ## Shift toward the rest-frame
                wave_res = wave / (1. + spez) # _res stands for rest-frame
                fwhm_res = fwhm / (1. + spez)  

                # Construct the SPS model 
                fwhm_gal   = {"lam" : wave_res, "fwhm": fwhm_res}
                sps        = lib.sps_lib(filename, velscale, fwhm_gal, wave_range=wave_range_temp)
                goodpixels = util.determine_goodpixels(np.log(wave_res), wave_range_temp)
                
                # Actual pPXF fitting
                vel        = 0.          # Since we already shifted to the rest-frame
                start      = [vel, 200.] # (km/s), starting guess for [V, sigma]
                time       = clock()   
                #print(redshift, np.min(lam_gal), np.max(lam_gal), len(galaxy), len(sps.templates))

                # Run pPXF
                pp = ppxf(sps.templates, nflux, noise, velscale, start,
                        goodpixels=goodpixels, plot=False, moments=4, trig=1,
                        degree=20, lam=wave_res, lam_temp=sps.lam_temp, quiet=False)
                
                result_str = np.array2string(pp.sol, separator=', ', formatter={'float': lambda x: str(x)})
                print(result_str)  # 출력: '[1, 2, 3, 4, 5]'

                f.write('Elapsed time in pPXF: %.2f s' % (clock() - time) + "\n")
                f.write(result_str)
                f.write("\n")  # 각 파일 처리 사이에 빈 줄 추가

                errors = pp.error*np.sqrt(pp.chi2)  # Assume the fit is good chi2/DOF=1
                redshift_fit = (1 + spez)*np.exp(pp.sol[0]/c) - 1     # eq. (5c) C23
                redshift_err = (1 + redshift_fit)*errors[0]/c               # eq. (5d) C23

                #f.write("Formal errors:\n")
                #f.write("     dV    dsigma   dh3      dh4\n")
                #f.write("".join("%8.2g" % f for f in errors))
                #f.write("\n")
                prec = int(1 - np.floor(np.log10(redshift_err)))  # two digits of uncertainty
                f.write(f"Best-fitting redshift z = {redshift_fit:#.{prec}f} \n"
                        f"+/- {redshift_err:#.{prec}f}\n")
                
                f.write(f"{file} processing done.\n")
                f.write(f"{file} processing done.\n")
                f.write(f"{file} processing done.\n")
                f.write("\n")  # 각 파일 처리 사이에 빈 줄 추가

spec-1938-53379-0110.fits processing start

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      -287        69    -0.033     0.036
chi2/DOF: 47.68; DOF: 3599; degree = 20; mdegree = 0
method = capfit; Jac calls: 8; Func calls: 46; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/150
[-287.3738257012113, 68.59758918132465, -0.033080628604325396,
 0.036391966673299206]
spec-1937-53388-0453.fits processing start

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:      2000       626     0.300     0.300
chi2/DOF: 11.90; DOF: 3600; degree = 20; mdegree = 0
method = capfit; Jac calls: 25; Func calls: 127; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 2/150
[1999.9999999999998, 626.2857260776346, 0.3, 0.3]
spec-1938-53379-0053.fits processing start

 Best Fit:       Vel     sigma        h3        h4
 comp.  0:        55       201     0.001     0.037
chi2/DOF: 3.235; DOF: 3595; degree = 20; mdegree = 0
method = capfit; Jac calls: 3; Func